# e) Power and Energy Model

Calculates power and energy over the historic daily flow series.

## Procedure

For each day in the series:
- Apply the selected Environmental Assurance Categories for each channel
- Determine the EWR flows for each channel
- Calculate the canal flow
- Calculate the headpond and tailrace levels
- Calculate headlosses in the left channel and canal
- Aportion canal flows amongst the turbines
- Calculate headlosses in the turbines and net head
- Calculate turbine and generator efficiency
- Calculate power and energy

## Inputs

| Data                       | Source                                        | Description                                 |
|----------------------------|-----------------------------------------------|---------------------------------------------|
| ewr_section_categories.csv |                                               |  EWR Assurance Category Sets            |
| ngonye_flow_daily_ewrx.csv | Notebook: d_ewr_setup             |  Daily flow data with the floww exceddance values (each 10 days) |
| headloss_leftchannel.csv   | Mott MacDonald - Ngonye Falls Hydropower Project - 2018 Feasibility Study Update - Final Report Version D | Headloss in the left channel by flow  |
| headloss_canal.csv         | Mott MacDonald - Ngonye Falls Hydropower Project - 2018 Feasibility Study Update - Final Report Version D | Headloss in the canal by flow |









## Libraries and Setup

In [429]:
import numpy as np
import pandas as pd
from scipy import interpolate
import math

## Parameters

In [430]:
input_data='./input_data/'
output_data='./output_data/'

In [431]:
model_name='Base Case'#MW153#MW144 'MW162'#'Fixed Tailwater'#'Base Case' Headpond50cm EWRAllCs
dryrun=False #Dont write output files

In [432]:
models=pd.read_csv(input_data + "models.csv").set_index('ModelName')
model=models.loc[model_name]

out_prefix=model['OutputPrefix']#'base'

category_set_name=model['EWRCatSet']#'Recommendation 1'
headpond_lift=model['HeadpondLift']#0
plant_capacity=model['Capacity']#180
canal_capacity=plant_capacity/180*1100#1100
fixed_tailwater_level=model['FixedTailwater']#False #970 

model


OutputPrefix                  base
Description              Base Case
EWRCatSet         Recommendation 1
HeadpondLift                     0
Capacity                       180
FixedTailwater                   0
Mean                        832123
P95                         717411
P90                         754939
P75                         791547
P50                         838318
P25                         875947
P10                         921510
P5                          957737
StDev                      82094.8
CoefVar                  0.0986571
Max                    1.01828e+06
Min                         493500
CapFactor                 0.527729
CalYearMean                 832218
CalYearStDev               78428.1
CalYearCoefVar           0.0942398
Name: Base Case, dtype: object

## Load Daily Data

Load the daily time series which includes the exceedance values used for calculating daily EWRs.

In [433]:
daily=pd.read_csv(output_data + "ngonye_flow_daily_ewrx.csv")
daily=daily.set_index(pd.to_datetime(daily['Date'],format="%Y-%m-%d"))
daily=daily.drop('Date',axis=1)
daily=daily.astype({'WaterWeek': 'int32'})
daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 33968 entries, 1924-10-01 to 2017-09-30
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   LaggedDate        33968 non-null  object 
 1   VicFalls          33968 non-null  float64
 2   Conversion        33968 non-null  float64
 3   Flow              33968 non-null  float64
 4   Exceedance        33968 non-null  float64
 5   Year              33968 non-null  int64  
 6   Month             33968 non-null  int64  
 7   Day               33968 non-null  int64  
 8   MonthId           33968 non-null  float64
 9   WaterYear         33968 non-null  int64  
 10  WaterMonth        33968 non-null  int64  
 11  WaterDay          33968 non-null  int64  
 12  WaterWeek         33968 non-null  int32  
 13  Volume            33968 non-null  float64
 14  EWRRefExceedance  33968 non-null  float64
dtypes: float64(7), int32(1), int64(6), object(1)
memory usage: 4.0+ MB


## EWR Assurance Category Sets
Load the available EWR assurance category sets

In [434]:
category_sets=pd.read_csv(input_data + "ewr_section_categories.csv")
category_sets

,Section,Recommendation 1,All Cs,All Bs,All As
0,A,B,C,B,A
1,C,B,C,B,A
2,D,B,C,B,A
3,E,B,C,B,A
4,FG,C,C,B,A


## EWR Assurance Category Set
Select the category set given as a variable in  the setup above

In [435]:
category_set=category_sets.loc[:,['Section',category_set_name]]
category_set=category_set.rename(columns={category_set_name:'Category'})
category_set

,Section,Category
0,A,B
1,C,B
2,D,B
3,E,B
4,FG,C


Load the EWR Flow Sets which specify the required EWRs by assurance category, channel section and calendar month

In [436]:
ewr_flow_sets=pd.read_csv(input_data + "ewr_flow_sets.csv")
ewr_flow_sets=ewr_flow_sets.set_index(['Section','Category'])
ewr_flow_sets

Month  Wet  Mod_Wet  Normal  Mod_Dry  Dry
Section Category                                           
A       A            10   75       70      60       47   38
        A            11   73       69      58       45   37
        A            12   77       72      62       48   40
        A             1   83       77      66       52   46
        A             2   90       81      69       56   51
...                 ...  ...      ...     ...      ...  ...
FG      D             5   76       76      66       49   42
        D             6   70       57      46       39   31
        D             7   64       53      41       31   27
        D             8   51       43      32       25   22
        D             9   36       31      23       17   15

[240 rows x 6 columns]

## EWR Flow Sets
Select the EWR Flow Set for each channel section according to the Category Set chosen above (eg *Recommendation 1*)

In [437]:
ewrs=category_set.join(ewr_flow_sets,on=['Section','Category']).set_index(['Section','Month'])
ewrs_A=ewrs.loc['A']
ewrs_C=ewrs.loc['C']
ewrs_D=ewrs.loc['D']
ewrs_E=ewrs.loc['E']
ewrs_FG=ewrs.loc['FG']

ewrs_A


,Category,Wet,Mod_Wet,Normal,Mod_Dry,Dry
Month,,,,,,
10,B,44,41,37,31,24
11,B,42,40,36,29,23
12,B,44,42,38,31,26
1,B,49,46,41,34,29
2,B,55,49,43,36,31
3,B,68,59,49,41,36
4,B,84,70,54,45,40
5,B,71,65,55,47,42
6,B,67,62,53,45,39


## EWR Flow Bands
Annotate the daily flow series with the EWR Flow Band (Wet, Very Wet, Dry etc) - a lookup against the EWR Exceedance

In [438]:
def flowBandNo(exceed):
    if exceed <= 0.1:
        return 1
    elif  exceed <= 0.4:
        return 2
    elif  exceed <= 0.7:
        return 3
    elif  exceed <= 0.9:
        return 4
    else:
        return 5
    
def flowBandLabel(bandNo):
    if bandNo == 1:
        return 'Wet'
    elif  bandNo == 2:
        return 'Mod_Wet'
    elif  bandNo == 3:
        return 'Normal'
    elif  bandNo ==4:
        return 'Mod_Dry'
    else:
        return 'Dry'    
    
def ewrLookup(section, band, month):
    return ewrs.loc[(section,month)][band]

In [439]:

daily['EWRBandNo']=daily.apply(lambda x: flowBandNo(x['EWRRefExceedance']),axis=1) 
daily['EWRBandLabel']=daily.apply(lambda x: flowBandLabel(x['EWRBandNo']),axis=1) 

## EWR Flows

In [440]:

daily['EWRChannelA']=ewrs_A.lookup(daily['Month'],daily['EWRBandLabel'])
daily['EWRChannelC']=ewrs_C.lookup(daily['Month'],daily['EWRBandLabel'])
daily['EWRChannelD']=ewrs_D.lookup(daily['Month'],daily['EWRBandLabel'])
daily['EWRChannelE']=ewrs_E.lookup(daily['Month'],daily['EWRBandLabel'])
daily['EWRChannelFG']=ewrs_FG.lookup(daily['Month'],daily['EWRBandLabel'])


In [441]:
daily['EWRTotal']=daily['EWRChannelA']+daily['EWRChannelC']+daily['EWRChannelD']+daily['EWRChannelE']+daily['EWRChannelFG']
daily['EWRProportion']=daily['EWRTotal']/daily['Flow']

## Spill and channel flows

In [442]:
daily['FlowCanal']=np.where(daily['Flow']-daily['EWRTotal']>canal_capacity,canal_capacity,daily['Flow']-daily['EWRTotal'])
daily['FlowSpill']=np.where(daily['Flow']-daily['EWRTotal']<=canal_capacity,0,daily['Flow']-daily['EWRTotal']-canal_capacity)

daily['SpillChannelA']=daily['FlowSpill']*(daily['EWRChannelA']/daily['EWRTotal'])
daily['SpillChannelC']=daily['FlowSpill']*(daily['EWRChannelC']/daily['EWRTotal'])
daily['SpillChannelD']=daily['FlowSpill']*(daily['EWRChannelD']/daily['EWRTotal'])
daily['SpillChannelE']=daily['FlowSpill']*(daily['EWRChannelE']/daily['EWRTotal'])
daily['SpillChannelFG']=daily['FlowSpill']*(daily['EWRChannelFG']/daily['EWRTotal'])

daily['FlowChannelA']=daily['SpillChannelA']+daily['EWRChannelA']
daily['FlowChannelC']=daily['SpillChannelC']+daily['EWRChannelC']
daily['FlowChannelD']=daily['SpillChannelD']+daily['EWRChannelD']
daily['FlowChannelE']=daily['SpillChannelE']+daily['EWRChannelE']

daily['TmpA']=np.where(daily['FlowChannelA']>100,daily['FlowChannelA']-100,0)
daily['SpillChannelA']=daily['SpillChannelA']-daily['TmpA']
daily['FlowChannelA']=daily['FlowChannelA']-daily['TmpA']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpA']

daily['TmpC']=np.where(daily['FlowChannelC']>18,daily['FlowChannelC']-18,0)
daily['SpillChannelC']=daily['SpillChannelC']-daily['TmpC']
daily['FlowChannelC']=daily['FlowChannelC']-daily['TmpC']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpC']

daily['TmpD']=np.where(daily['FlowChannelD']>25,daily['FlowChannelD']-25,0)
daily['SpillChannelD']=daily['SpillChannelD']-daily['TmpD']
daily['FlowChannelD']=daily['FlowChannelD']-daily['TmpD']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpD']

daily['TmpE']=np.where(daily['FlowChannelE']>80,daily['FlowChannelE']-80,0)
daily['SpillChannelE']=daily['SpillChannelE']-daily['TmpE']
daily['FlowChannelE']=daily['FlowChannelE']-daily['TmpE']
daily['SpillChannelFG']=daily['SpillChannelFG']+daily['TmpE']

daily['FlowChannelFG']=daily['SpillChannelFG']+daily['EWRChannelFG']
daily=daily.drop(['TmpA','TmpC','TmpD','TmpE'],axis=1)

daily['FlowLeftChannel']=daily['FlowChannelA']+daily['FlowChannelC']+daily['FlowChannelD']+daily['FlowCanal']


In [443]:

daily['Turbines']=daily.apply(lambda x: math.ceil(x['FlowCanal'] / (canal_capacity/4)),axis=1)
daily.loc[daily['FlowCanal']<=plant_capacity/180*50,'Turbines']=0

In [444]:
daily['FlowTurbine1']=np.where(daily['Turbines']>=1,daily['FlowCanal']/daily['Turbines'],np.nan)
daily['FlowTurbine2']=np.where(daily['Turbines']>=2,daily['FlowCanal']/daily['Turbines'],np.nan)
daily['FlowTurbine3']=np.where(daily['Turbines']>=3,daily['FlowCanal']/daily['Turbines'],np.nan)
daily['FlowTurbine4']=np.where(daily['Turbines']==4,daily['FlowCanal']/daily['Turbines'],np.nan)

## Levels, headloss and head
\begin{equation*}
Headloss Powerhouse=0.00000702Q^2 + 0.00002226Q - 0.00207422
\end{equation*}

\begin{equation*}
Tailwater Level=6.0977*log(Q)^2-27.598*log(Q)+995.82
\end{equation*}

\begin{equation}
  Headpond Level=\begin{cases}
    990.0, & \text{if $Q<5000$}.\\
    0.000081307*Q+989.6, & \text{if $Q<7500$}.\\
    0.000240*Q+988.4, & \text{otherwise}.
  \end{cases}
\end{equation}


In [445]:
def headpondLevel(flow):
    if flow<5000:
        return 990.0 + headpond_lift
    elif flow<7500:
        return 0.000081307*flow+989.6 + headpond_lift
    else:
        return 0.000240*flow+988.4 + headpond_lift

In [446]:
def tailwaterLevel(flow):
    if fixed_tailwater_level>0:
        return fixed_tailwater_level
    else:
        log_flow=math.log10(flow)
        if log_flow>=3.8:
            return -21.429*log_flow**2+182.03*log_flow+597.15   
        elif log_flow>=2.3:
            return 6.5299*log_flow**2-30.155*log_flow+999.52   
        else:
            return -2.6828*log_flow**2+11.265*log_flow+952.68

In [447]:
daily['LevelTailwater']=daily['Flow'].apply(tailwaterLevel)
daily['LevelHeadpond']=daily['Flow'].apply(headpondLevel)

In [448]:
headloss_leftchannel=pd.read_csv(input_data + "headloss_leftchannel.csv")
fHeadlossLeftChannel = interpolate.interp1d(headloss_leftchannel['FlowLeftChannel'], headloss_leftchannel['HeadlossLeftChannel'])
daily['HeadlossLeftChannel']=daily['FlowLeftChannel'].apply(fHeadlossLeftChannel)


In [449]:
headloss_canal=pd.read_csv(input_data + "headloss_canal.csv")
fHeadlossCanal = interpolate.interp1d(headloss_canal['FlowCanal'], headloss_canal['HeadlossCanal'])

daily['HeadlossCanal']=daily['FlowCanal'].apply(fHeadlossCanal)
daily['LevelForebay']=daily['LevelHeadpond']-daily['HeadlossLeftChannel']-daily['HeadlossCanal']

In [450]:
def turbineHeadloss(flow):
    if np.isnan(flow):
        return np.nan
    elif flow==0:
        return 0
    else:
        return 0.00000702*flow**2+0.00002226*flow-0.00207422

In [451]:
daily['HeadlossTurbine1']=daily['FlowTurbine1'].apply(turbineHeadloss)
daily['HeadlossTurbine2']=daily['FlowTurbine2'].apply(turbineHeadloss)
daily['HeadlossTurbine3']=daily['FlowTurbine3'].apply(turbineHeadloss)
daily['HeadlossTurbine4']=daily['FlowTurbine4'].apply(turbineHeadloss)

In [452]:
daily['HeadTurbine1']=np.where(daily['FlowTurbine1']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine1'],np.nan)
daily['HeadTurbine2']=np.where(daily['FlowTurbine2']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine2'],np.nan)
daily['HeadTurbine3']=np.where(daily['FlowTurbine3']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine3'],np.nan)
daily['HeadTurbine4']=np.where(daily['FlowTurbine4']>0,daily['LevelForebay']-daily['LevelTailwater']-daily['HeadlossTurbine4'],np.nan)
daily.loc[daily['HeadTurbine1']<=10,'Turbines']=0

## Turbine and generator efficiency

In [453]:
def turbineEfficiency(q_actual,h):
    
    #Scale flow to installed capacity
    q=q_actual*(180/plant_capacity)
    
    if np.isnan(h) or np.isnan(q):
        return np.nan
    
    if h<=10:
        return 0
    
    if q==0:
        return 0
    elif q<53.5:
        return 0.77
    elif q<55:
        return 0.78
    elif q<58.5:
        return 0.79
    elif q<62:
        return 0.80
    elif q<64:
        return 0.81
    elif q<66.5:
        return 0.82
    elif q<70:
        return 0.83
    elif q<72:
        return 0.84
    elif q<74.5:
        return 0.85
    elif q<82.5:
        return 0.86
    elif q<91.5:
        return 0.87
    elif q<100.5:
        return 0.88
    elif q<109.72:
        return 0.89
    elif q<121.77:
        return 0.90
    elif q<146.8:
        return 0.91
    elif q<169.545:
        return 0.92
    elif q<173.1:
        if h<24.65:
            return 0.92
        else:
            return 0.93
    elif q<199.8:
        return 0.93
    elif q<211.2:
        if h<23:
            return 0.93
        else:
            return 0.94
    elif q<247.05:
        return 0.94
    elif q<250:
        if h<24.16:
            return 0.94
        else:
            return 0.95
    elif q<260:
        if h<23.6:
            return 0.94
        else:
            return 0.95
    elif q<275:
        if h<22.5:
            return 0.94
        else:
            return 0.95
    else:
        return -0.0015*h**2 + 0.0565*h + 0.4

In [454]:
daily['EffTurbine1']=daily.apply(lambda x: turbineEfficiency(x['FlowTurbine1'],x['HeadTurbine1']),axis=1)
daily['EffTurbine2']=daily.apply(lambda x: turbineEfficiency(x['FlowTurbine2'],x['HeadTurbine2']),axis=1)
daily['EffTurbine3']=daily.apply(lambda x: turbineEfficiency(x['FlowTurbine3'],x['HeadTurbine3']),axis=1)
daily['EffTurbine4']=daily.apply(lambda x: turbineEfficiency(x['FlowTurbine4'],x['HeadTurbine4']),axis=1)

In [455]:
daily['LoadFactorTurbine1']=daily['FlowTurbine1']/250*(plant_capacity/180)
daily['LoadFactorTurbine2']=daily['FlowTurbine2']/250*(plant_capacity/180)
daily['LoadFactorTurbine3']=daily['FlowTurbine3']/250*(plant_capacity/180)
daily['LoadFactorTurbine4']=daily['FlowTurbine4']/250*(plant_capacity/180)

In [456]:
def generatorEff(load_fact):
    if np.isnan(load_fact):
        return np.nan
    
    if load_fact==0:
        return 0
    elif load_fact<=0.45:
        return 0.96
    elif load_fact<=0.85:
        return 0.97
    else:
        return 0.98

In [457]:
daily['EffGen1']=daily['LoadFactorTurbine1'].apply(generatorEff)
daily['EffGen2']=daily['LoadFactorTurbine2'].apply(generatorEff)
daily['EffGen3']=daily['LoadFactorTurbine3'].apply(generatorEff)
daily['EffGen4']=daily['LoadFactorTurbine4'].apply(generatorEff)

## Power and energy

In [458]:
daily['PowerTurbine1']=daily['FlowTurbine1']*daily['HeadTurbine1']*daily['EffTurbine1']*daily['EffGen1']*9.81/1000
daily['PowerTurbine2']=daily['FlowTurbine2']*daily['HeadTurbine2']*daily['EffTurbine2']*daily['EffGen2']*9.81/1000
daily['PowerTurbine3']=daily['FlowTurbine3']*daily['HeadTurbine3']*daily['EffTurbine3']*daily['EffGen3']*9.81/1000
daily['PowerTurbine4']=daily['FlowTurbine4']*daily['HeadTurbine4']*daily['EffTurbine4']*daily['EffGen4']*9.81/1000
daily['Power']=np.nan_to_num(daily['PowerTurbine1'])+np.nan_to_num(daily['PowerTurbine2'])+np.nan_to_num(daily['PowerTurbine3'])+np.nan_to_num(daily['PowerTurbine4'])
daily['Energy']=daily['Power']*24
daily['PerfMwPerCumec']=daily['Power']/daily['Flow']

## Summarise
Aggregate into months, years, calendar months etc

In [459]:

monthly=daily.groupby("MonthId").mean().drop(['VicFalls','Conversion','Day','WaterDay','WaterWeek','Volume','EWRBandNo','EWRRefExceedance','Energy'],axis=1)

monthly=monthly.join(daily.groupby("MonthId").agg(    
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),   
   Turbines_max=('Turbines', 'max'),    
   Turbines_min=('Turbines', 'min'),   
   Energy=('Energy', 'sum'),
))
monthly['Days']=monthly.apply(lambda x: pd.Period(year=x['Year'],month=x['Month'],freq='D').days_in_month,axis=1)
monthly['CapFactor']=monthly['Energy']/(plant_capacity*24*monthly['Days'])

monthly.tail(12)

,Flow,Exceedance,Year,Month,WaterYear,WaterMonth,EWRChannelA,EWRChannelC,EWRChannelD,EWRChannelE,...,PowerTurbine4,Power,PerfMwPerCumec,Power_max,Power_min,Turbines_max,Turbines_min,Energy,Days,CapFactor
MonthId,,,,,,,,,,,,,,,,,,,,,
2016.10,186.117272,0.973290,2016.0,10.0,2016.0,1.0,31.000000,4.000000,3.000000,2.000000,...,NaN,24.332111,0.130268,29.616807,19.265120,1,1,18103.090598,31,0.135178
2016.11,186.562065,0.968767,2016.0,11.0,2016.0,2.0,25.000000,4.000000,2.000000,2.000000,...,NaN,28.458823,0.150451,39.837904,18.354687,1,1,20490.352445,30,0.158105
2016.12,369.487694,0.701516,2016.0,12.0,2016.0,3.0,29.387097,4.677419,3.000000,2.000000,...,NaN,64.627935,0.174405,106.746810,39.575231,2,1,48083.183741,31,0.359044
2017.01,851.896141,0.394581,2017.0,1.0,2016.0,4.0,44.387097,6.677419,5.354839,3.677419,...,42.871572,139.045177,0.162158,177.265268,67.172040,4,2,103449.611851,31,0.772473
2017.02,1236.957587,0.273679,2017.0,2.0,2016.0,5.0,49.000000,8.000000,7.000000,5.000000,...,45.300024,181.200095,0.148101,192.306328,171.035855,4,4,121766.464031,28,1.006667
2017.03,2420.771572,0.145774,2017.0,3.0,2016.0,6.0,49.000000,9.000000,9.000000,9.000000,...,38.819992,155.279968,0.075089,181.298207,117.687033,4,4,115528.295911,31,0.862666
2017.04,3419.680887,0.063633,2017.0,4.0,2016.0,7.0,70.000000,14.000000,18.000000,25.000000,...,31.971398,127.885591,0.038018,142.568515,116.887838,4,4,92077.625765,30,0.710476
2017.05,2194.251076,0.156387,2017.0,5.0,2016.0,8.0,55.000000,13.000000,13.000000,17.000000,...,40.090687,160.362747,0.075410,177.436189,144.100481,4,4,119309.883944,31,0.890904
2017.06,1117.935234,0.312067,2017.0,6.0,2016.0,9.0,53.000000,12.000000,12.000000,13.000000,...,44.600389,156.045711,0.141604,188.863966,95.686797,4,2,112352.912049,30,0.866921


In [460]:
yearly=daily.groupby("WaterYear").agg(
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
)
yearly['CapFactor']=yearly['Energy']/(plant_capacity*24*365)


In [461]:
yearly['Prop4Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==4,1,0))/365)
yearly['Prop3Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==3,1,0))/365)
yearly['Prop2Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==2,1,0))/365)
yearly['Prop1Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==1,1,0))/365)
yearly['Prop0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0))/365)
yearly['Days0Units']=daily[['WaterYear','Turbines']].groupby('WaterYear').agg(lambda x: sum(np.where(x==0,1,0)))


In [462]:
edc=pd.DataFrame(index=np.arange(0,1.05,0.05))
edc['Energy']=np.percentile(yearly['Energy'],(1-edc.index)*100)
edc

,Energy
0.00,1.018276e+06
0.05,9.577369e+05
0.10,9.215099e+05
0.15,9.019275e+05
0.20,8.880480e+05
0.25,8.759469e+05
0.30,8.669487e+05
0.35,8.561403e+05
0.40,8.518980e+05
0.45,8.429148e+05


In [463]:
yearly['Exceedance']=pd.merge_asof(yearly.reset_index().sort_values('Energy'),edc.reset_index().sort_values('Energy'),left_on='Energy',right_on='Energy').set_index('WaterYear')['index']

In [464]:
calyearly=daily.groupby("Year").agg(  
   Energy=('Energy', 'sum'),
).drop([1924,2017],axis=0)
calyearly['CapFactor']=calyearly['Energy']/(180*24*365)
calyearly.head(5)

,Energy,CapFactor
Year,,
1925,853834.972289,0.541499
1926,777044.104101,0.492798
1927,853200.496589,0.541096
1928,829378.408235,0.525988
1929,753348.824234,0.477771


In [465]:
monthly_day=daily.loc[daily['Day']==15]

In [466]:
daily_slim=daily.loc[:,
                     ['Year',
                      'Month',
                      'Day',
                      'WaterYear',
                      'Flow',
                      'EWRRefExceedance',
                      'EWRBandNo',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'Turbines',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',    
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion': 2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,    
                    })
daily_slim.tail(12)

,Year,Month,Day,WaterYear,Flow,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,...,EffTurbine1,EffTurbine2,EffTurbine3,EffTurbine4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy
Date,,,,,,,,,,,,,,,,,,,,,
2017-09-19,2017,9,19,2016,216,0.81,4,34,6,4,...,0.91,0.0,0.0,0.0,27.2,0.0,0.0,0.0,27.2,653
2017-09-20,2017,9,20,2016,213,0.81,4,34,6,4,...,0.91,0.0,0.0,0.0,26.5,0.0,0.0,0.0,26.5,636
2017-09-21,2017,9,21,2016,213,0.82,4,34,6,4,...,0.91,0.0,0.0,0.0,26.5,0.0,0.0,0.0,26.5,636
2017-09-22,2017,9,22,2016,209,0.82,4,34,6,4,...,0.90,0.0,0.0,0.0,25.5,0.0,0.0,0.0,25.5,611
2017-09-23,2017,9,23,2016,203,0.82,4,34,6,4,...,0.90,0.0,0.0,0.0,23.9,0.0,0.0,0.0,23.9,572
2017-09-24,2017,9,24,2016,203,0.82,4,34,6,4,...,0.90,0.0,0.0,0.0,23.8,0.0,0.0,0.0,23.8,572
2017-09-25,2017,9,25,2016,203,0.82,4,34,6,4,...,0.90,0.0,0.0,0.0,23.8,0.0,0.0,0.0,23.8,572
2017-09-26,2017,9,26,2016,203,0.82,4,34,6,4,...,0.90,0.0,0.0,0.0,23.8,0.0,0.0,0.0,23.8,572
2017-09-27,2017,9,27,2016,203,0.82,4,34,6,4,...,0.90,0.0,0.0,0.0,23.8,0.0,0.0,0.0,23.8,572


In [467]:
weekly=daily.groupby(["WaterYear","WaterWeek"]).mean().drop(['Month','Year','MonthId','WaterMonth','VicFalls','Conversion','Day','WaterDay','Volume','EWRBandNo','EWRRefExceedance','Turbines','Energy'],axis=1)
weekly=weekly.join(daily.groupby(["WaterYear","WaterWeek"]).agg(    
   Flow_max=('Flow', 'max'),
   Flow_min=('Flow', 'min'), 
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_min=('EWRProportion', 'min'),
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_min=('HeadTurbine1', 'min'),    
   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_min=('LevelTailwater', 'min'),  
   Power_max=('Power', 'max'),
   Power_min=('Power', 'min'),    
   Energy=('Energy', 'sum'),
))
weekly=weekly.join(daily[["WaterYear","WaterWeek","Flow"]].groupby(["WaterYear","WaterWeek"]).count().rename(columns={'Flow':'Count'}))
weekly['CapFactor']=weekly['Energy']/(180*24*weekly['Count'])
weekly=weekly.join(daily.reset_index('Date')[["WaterYear","WaterWeek","Date"]].groupby(["WaterYear","WaterWeek"]).first().rename(columns={'Date':'StartDate'}))

weekly.tail(12)

Flow  Exceedance  EWRChannelA  EWRChannelC  \
WaterYear WaterWeek                                                     
2016      42         451.673869    0.594143    50.000000    10.000000   
          43         411.631943    0.638429    50.000000    10.000000   
          44         377.984000    0.677571    43.714286     8.285714   
          45         345.089425    0.718000    39.000000     7.000000   
          46         322.320457    0.753571    39.000000     7.000000   
          47         296.353596    0.789429    39.000000     7.000000   
          48         272.698436    0.833000    38.285714     6.857143   
          49         250.364895    0.874143    34.000000     6.000000   
          50         228.116819    0.912143    34.000000     6.000000   
          51         215.508577    0.932000    34.000000     6.000000   
          52         203.039212    0.949714    34.000000     6.000000   
          53         198.229576    0.958000    34.000000     6.000000   

                     EWRChannelD  EWRChannelE  EWRChannelFG    EWRTotal  \
WaterYear WaterWeek                                                       
2016      42            9.000000     9.000000    108.000000  186.000000   
          43            9.000000     9.000000    108.000000  186.000000   
          44            6.714286     6.142857     80.571429  145.428571   
          45            5.000000     4.000000     60.000000  115.000000   
          46            5.000000     4.000000     60.000000  115.000000   
          47            5.000000     4.000000     60.000000  115.000000   
          48            4.857143     3.857143     57.714286  111.571429   
          49            4.000000     3.000000     44.000000   91.000000   
          50            4.000000     3.000000     44.000000   91.000000   
          51            4.000000     3.000000     44.000000   91.000000   
          52            4.000000     3.000000     44.000000   91.000000   
          53            4.000000     3.000000     44.000000   91.000000   

                     EWRProportion   FlowCanal  ...  HeadTurbine1_max  \
WaterYear WaterWeek                             ...                     
2016      42              0.412191  265.673869  ...         24.154500   
          43              0.452114  225.631943  ...         24.263769   
          44              0.383130  232.555429  ...         24.397698   
          45              0.333428  230.089425  ...         24.512254   
          46              0.357011  207.320457  ...         24.680765   
          47              0.388246  181.353596  ...         24.815893   
          48              0.408941  161.127008  ...         24.936125   
          49              0.363667  159.364895  ...         24.990447   
          50              0.399211  137.116819  ...         25.067046   
          51              0.422374  124.508577  ...         25.096632   
          52              0.448189  112.039212  ...         25.111062   
          53              0.459064  107.229576  ...         25.426292   

                     HeadTurbine1_min  LevelTailwater_max  LevelTailwater_min  \
WaterYear WaterWeek                                                             
2016      42                23.851344          965.578247          965.399573   
          43                24.039147          965.383886          965.265004   
          44                24.170282          965.240711          965.106315   
          45                24.350218          965.085857          965.005058   
          46                24.528270          964.997026          964.920026   
          47                24.695029          964.912737          964.851233   
          48                24.825439          964.846410          964.785468   
          49                24.895196          964.781163          964.740304   
          50                25.009894          964.732714          964.713477   
          51                25.067046  

In [468]:
weekly.head(4)

Flow  Exceedance  EWRChannelA  EWRChannelC  \
WaterYear WaterWeek                                                     
1924      1          111.133124       0.999    37.000000     5.000000   
          2          111.133124       0.999    29.571429     4.428571   
          3          112.652352       0.999    24.000000     4.000000   
          4          125.199744       0.998    24.000000     4.000000   

                     EWRChannelD  EWRChannelE  EWRChannelFG   EWRTotal  \
WaterYear WaterWeek                                                      
1924      1             3.000000          2.0     50.000000  97.000000   
          2             2.428571          2.0     32.857143  71.285714   
          3             2.000000          2.0     20.000000  52.000000   
          4             2.000000          2.0     20.000000  52.000000   

                     EWRProportion  FlowCanal  ...  HeadTurbine1_max  \
WaterYear WaterWeek                            ...                     
1924      1               0.872827  14.133124  ...               NaN   
          2               0.641444  39.847410  ...         25.408305   
          3               0.462069  60.652352  ...         25.408305   
          4               0.415353  73.199744  ...         25.384164   

                     HeadTurbine1_min  LevelTailwater_max  LevelTailwater_min  \
WaterYear WaterWeek                                                             
1924      1                       NaN          964.497633          964.497633   
          2                 25.408305          964.497633          964.497633   
          3                 25.387623          964.504830          964.497633   
          4                 25.380968          964.505351          964.505254   

                     Power_max  Power_min       Energy  Count  CapFactor  \
WaterYear WaterWeek                                                        
1924      1           0.000000   0.000000     0.000000      7   0.000000   
          2          11.319748   0.000000  1086.695792      7   0.035936   
          3          13.845062  11.319748  1962.325169      7   0.064892   
          4          15.082851  14.630748  2498.682541      7   0.082628   

                     StartDate  
WaterYear WaterWeek             
1924      1         1924-10-01  
          2         1924-10-08  
          3         1924-10-15  
          4         1924-10-22  

[4 rows x 71 columns]

In [469]:
weekly_slim=weekly.loc[:,
                     [
                      'StartDate',
                      'Flow',
                      'EWRChannelA',
                      'EWRChannelC',
                      'EWRChannelD',
                      'EWRChannelE',
                      'EWRChannelFG',
                      'EWRTotal',
                      'EWRProportion',
                      'FlowCanal',
                      'FlowSpill',
                      'SpillChannelA',
                      'SpillChannelC',
                      'SpillChannelD',
                      'SpillChannelE',
                      'SpillChannelFG',
                      'FlowChannelA',
                      'FlowChannelC',
                      'FlowChannelD',
                      'FlowChannelE',
                      'FlowChannelFG',
                      'FlowLeftChannel',
                      'FlowTurbine1',
                      'FlowTurbine2',
                      'FlowTurbine3',
                      'FlowTurbine4',
                      'LevelHeadpond',
                      'LevelTailwater',
                      'HeadlossLeftChannel',
                      'HeadlossCanal',
                      'LevelForebay',
                      'HeadTurbine1',
                      'HeadTurbine2',
                      'HeadTurbine3',
                      'HeadTurbine4',
                      'EffTurbine1',
                      'EffTurbine2',
                      'EffTurbine3',
                      'EffTurbine4',
                      'PowerTurbine1',
                      'PowerTurbine2',
                      'PowerTurbine3',
                      'PowerTurbine4',
                      'Power',
                      'Energy'
                     ]
                    ].fillna(0).astype({
                        'Flow': 'int32',
                        'FlowCanal': 'int32',
                        'FlowSpill': 'int32',
                        'EWRChannelA': 'int32',
                        'EWRChannelC': 'int32',
                        'EWRChannelD': 'int32',    
                        'EWRChannelE': 'int32',
                        'EWRChannelFG': 'int32',
                        'EWRTotal': 'int32',
                        'SpillChannelA': 'int32',
                        'SpillChannelC': 'int32',
                        'SpillChannelD': 'int32',    
                        'SpillChannelE': 'int32',
                        'SpillChannelFG': 'int32',
                        'FlowChannelA': 'int32',
                        'FlowChannelC': 'int32',
                        'FlowChannelD': 'int32',    
                        'FlowChannelE': 'int32',
                        'FlowChannelFG': 'int32',
                        'FlowLeftChannel': 'int32',
                        'FlowTurbine1': 'int32',
                        'FlowTurbine2': 'int32',
                        'FlowTurbine3': 'int32',
                        'FlowTurbine4': 'int32',
                        'Energy': 'int32',
                        
                    }).round({
                            'EWRProportion':2,
                            'LevelHeadpond': 1,
                            'LevelTailwater': 1,
                            'HeadlossLeftChannel': 2,
                            'HeadlossCanal': 2,
                            'LevelForebay': 1,
                            'PowerTurbine1': 1,
                            'PowerTurbine2': 1,
                            'PowerTurbine3': 1,
                            'PowerTurbine4': 1,
                            'Power': 1,
                            'HeadTurbine1':1,
                            'HeadTurbine2':1,
                            'HeadTurbine3':1,
                            'HeadTurbine4':1,
                            'EffTurbine1':2,
                            'EffTurbine2':2,
                            'EffTurbine3':2,
                            'EffTurbine4':2,
                    })

In [471]:
calmonthly=pd.DataFrame({'WaterMonth': [1,2,3,4,5,6,7,8,9,10,11,12],'MonthName': ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep'], 'Month':[10,11,12,1,2,3,4,5,6,7,8,9]})
calmonthly=calmonthly.set_index('WaterMonth')
calmonthly=calmonthly.join(monthly.groupby(["WaterMonth"]).agg(    
   EWRProportion_max=('EWRProportion', 'max'),
   EWRProportion_mean=('EWRProportion', 'mean'),
   EWRProportion_median=('EWRProportion', 'median'),
   EWRProportion_min=('EWRProportion', 'min'),
    
    EWRTotal_max=('EWRTotal', 'max'),
   EWRTotal_mean=('EWRTotal', 'mean'),
   EWRTotal_median=('EWRTotal', 'median'),
   EWRTotal_min=('EWRTotal', 'min'),   
    
   FlowCanal_max=('FlowCanal', 'max'),
   FlowCanal_mean=('FlowCanal', 'mean'),
   FlowCanal_median=('FlowCanal', 'median'),
   FlowCanal_min=('FlowCanal', 'min'),     
    
   HeadTurbine1_max=('HeadTurbine1', 'max'),
   HeadTurbine1_mean=('HeadTurbine1', 'mean'),
   HeadTurbine1_median=('HeadTurbine1', 'median'),
   HeadTurbine1_min=('HeadTurbine1', 'min'), 

   LevelTailwater_max=('LevelTailwater', 'max'),
   LevelTailwater_mean=('LevelTailwater', 'mean'),    
   LevelTailwater_median=('LevelTailwater', 'median'),    
   LevelTailwater_min=('LevelTailwater', 'min'),  
   LevelHeadpond_max=('LevelHeadpond', 'max'),
   LevelHeadpond_mean=('LevelHeadpond', 'mean'),    
   LevelHeadpond_median=('LevelHeadpond', 'median'),    
   LevelHeadpond_min=('LevelHeadpond', 'min'),  
   Turbines_mean=('Turbines', 'mean'),      
   Power_max=('Power', 'max'),
   Power_mean=('Power', 'mean'),
   Power_median=('Power', 'median'),
   Power_min=('Power', 'min'),    
   Energy_max=('Energy', 'max'),
   Energy_mean=('Energy', 'mean'),
   Energy_median=('Energy', 'median'),    
   Energy_min=('Energy', 'min'),
))
calmonthly['Energy_P95']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.05)
calmonthly['Energy_P90']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.10)
calmonthly['Energy_P75']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.25)
calmonthly['Energy_P25']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.75)
calmonthly['Energy_P10']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.90)
calmonthly['Energy_P5']=monthly[['WaterMonth','Energy']].groupby('WaterMonth').quantile(0.95)

calmonthly

,MonthName,Month,EWRProportion_max,EWRProportion_mean,EWRProportion_median,EWRProportion_min,EWRTotal_max,EWRTotal_mean,EWRTotal_median,EWRTotal_min,...,Energy_max,Energy_mean,Energy_median,Energy_min,Energy_P95,Energy_P90,Energy_P75,Energy_P25,Energy_P10,Energy_P5
WaterMonth,,,,,,,,,,,,,,,,,,,,,
1,Oct,10,0.580145,0.397072,0.399437,0.274238,120.0,93.623309,97.000000,52.0,...,50141.866536,23404.775648,21379.602066,6623.517173,16779.009746,18108.433907,18975.943328,26720.100987,31375.918331,36662.513294
2,Nov,11,0.403580,0.331341,0.335545,0.219146,114.0,89.311828,93.000000,50.0,...,62336.909091,29252.460345,26896.609032,15832.921537,19507.457746,21521.724743,23160.892636,32371.571058,41017.539738,45526.881415
3,Dec,12,0.288862,0.242016,0.242502,0.158090,126.0,100.352064,104.000000,59.0,...,100116.305839,52331.596533,48083.183741,27990.130687,36666.581580,38286.904373,42980.687016,60675.673061,68476.679660,84382.563406
4,Jan,1,0.222615,0.177630,0.181580,0.097584,150.0,117.763441,120.000000,73.0,...,128765.269889,83784.767802,76874.171942,42075.499445,56533.543761,62122.215361,69380.336152,97409.984090,117291.512884,120794.956948
5,Feb,2,0.188613,0.130120,0.140135,0.023962,197.0,139.328407,137.000000,85.0,...,127357.062086,98068.378571,98458.699746,4974.159183,71903.546145,78940.014106,89260.582672,111684.779980,119344.667302,121437.322364
6,Mar,3,0.183460,0.095537,0.091553,0.040247,315.0,194.510926,185.451613,107.0,...,137626.598241,103754.691855,111377.761914,0.000000,20056.050553,76523.889757,95089.476186,121691.011717,129830.224882,132882.701240
7,Apr,4,0.165319,0.085341,0.083222,0.049988,364.0,231.824373,217.000000,125.0,...,137636.032255,98333.409290,100034.857867,0.000000,58608.568134,73225.501599,87253.537128,115053.625636,129370.641139,133697.593752
8,May,5,0.223318,0.130912,0.128444,0.066398,340.0,265.271245,276.000000,147.0,...,139123.757918,114103.911897,116552.535609,44311.092642,87884.237453,91542.929350,106030.813041,126582.922198,131073.402343,133791.793886
9,Jun,6,0.369589,0.200097,0.194879,0.108743,293.0,212.774194,210.000000,119.0,...,126812.518857,105040.035001,114158.048248,32700.049047,50668.065668,71715.408666,95314.391801,121566.554261,123458.441953,124713.898354


In [472]:
#fdc=pd.DataFrame(index=np.arange(0,1.01,0.01))
#fdc=pd.merge_asof(fdc,daily.sort_values('Exceedance'),left_index=True,right_on='Exceedance').reset_index().set_index('Exceedance').drop(['Date','LaggedDate','VicFalls','Conversion','Year','Month','Day','MonthId','WaterYear','WaterDay','WaterMonth','WaterWeek'],axis=1)

#fdc=daily.reset_index().groupby(lambda x: np.round(x['Exceedance'],2)).mean()
daily['ExceedanceRnd']=np.round(daily['Exceedance'],2)
fdc=daily.reset_index().groupby('ExceedanceRnd').mean().drop(['VicFalls','Conversion','Year','Month','Day','MonthId','WaterYear','WaterDay','WaterMonth','WaterWeek'],axis=1)
fdc

,Flow,Exceedance,Volume,EWRRefExceedance,EWRBandNo,EWRChannelA,EWRChannelC,EWRChannelD,EWRChannelE,EWRChannelFG,...,EffGen2,EffGen3,EffGen4,PowerTurbine1,PowerTurbine2,PowerTurbine3,PowerTurbine4,Power,Energy,PerfMwPerCumec
ExceedanceRnd,,,,,,,,,,,,,,,,,,,,,
0.00,7594.902897,0.003034,0.656200,0.027727,1.051136,69.602273,13.278409,16.482955,20.210227,186.750000,...,0.98,0.98,0.98,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.01,5580.209657,0.010039,0.482130,0.082984,1.311475,72.475410,13.534426,17.488525,22.714754,191.859016,...,0.98,0.98,0.98,10.472819,10.472819,10.472819,10.472819,41.891275,1005.390605,0.007817
0.02,4801.054211,0.019973,0.414811,0.108374,1.387534,69.439024,12.915989,16.230352,20.447154,178.582656,...,0.98,0.98,0.98,24.634303,24.634303,24.634303,24.634303,98.537212,2364.893077,0.020619
0.03,4304.404444,0.029846,0.371901,0.146328,1.619672,67.550820,12.944262,16.032787,20.488525,170.960656,...,0.98,0.98,0.98,27.043952,27.043952,27.043952,27.043952,108.175807,2596.219366,0.025167
0.04,3931.296878,0.039886,0.339664,0.176180,1.737401,66.405836,12.904509,15.851459,20.403183,169.127321,...,0.98,0.98,0.98,29.008779,29.008779,29.008779,29.008779,116.035118,2784.842820,0.029540
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.96,199.590995,0.958409,0.017245,0.774358,3.937743,31.976654,4.712062,3.035019,2.089494,36.696498,...,NaN,NaN,NaN,26.241407,NaN,NaN,NaN,26.241407,629.793778,0.131508
0.97,193.907015,0.969858,0.016754,0.842789,4.281899,29.792285,4.430267,2.943620,2.065282,31.531157,...,NaN,NaN,NaN,26.904793,NaN,NaN,NaN,26.904793,645.715042,0.138762
0.98,181.492959,0.980640,0.015681,0.902596,4.545723,28.000000,4.395280,2.755162,2.035398,27.755162,...,NaN,NaN,NaN,25.253831,NaN,NaN,NaN,25.253831,606.091946,0.139138


In [ ]:
if dryrun==False:
    if out_prefix!='base':
        out_prefix="scenarios/" + out_prefix
    daily.to_csv(output_data+out_prefix+'_pe_daily.csv')
    monthly.to_csv(output_data+out_prefix+'_pe_monthly.csv')
    yearly.to_csv(output_data+out_prefix+'_pe_yearly.csv')
    edc.to_csv(output_data+out_prefix+'_pe_percentiles.csv')
    monthly_day.to_csv(output_data+out_prefix+'_pe_monthly_day.csv')
    daily_slim.to_csv(output_data+out_prefix+'_pe_daily_slim.csv')
    weekly.to_csv(output_data+out_prefix+'_pe_weekly.csv')
    weekly_slim.to_csv(output_data+out_prefix+'_pe_weekly_slim.csv')
    calmonthly.to_csv(output_data+out_prefix+'_pe_calmonthly.csv')
    fdc.to_csv(output_data+out_prefix+'_pe_fdc.csv')

In [ ]:
models.at[model_name,'Mean']=yearly['Energy'].mean()
models.at[model_name,'Max']=yearly['Energy'].max()
models.at[model_name,'Min']=yearly['Energy'].min()

models.at[model_name,'P95']=yearly['Energy'].quantile(0.05)
models.at[model_name,'P90']=yearly['Energy'].quantile(0.10)
models.at[model_name,'P75']=yearly['Energy'].quantile(0.25)
models.at[model_name,'P50']=yearly['Energy'].quantile(0.5)
models.at[model_name,'P25']=yearly['Energy'].quantile(0.75)
models.at[model_name,'P10']=yearly['Energy'].quantile(0.90)
models.at[model_name,'P5']=yearly['Energy'].quantile(0.95)

models.at[model_name,'StDev']=yearly['Energy'].std()
models.at[model_name,'CoefVar']=models.at[model_name,'StDev']/models.at[model_name,'Mean']

models.at[model_name,'CapFactor']=yearly['CapFactor'].mean()

models.at[model_name,'CalYearMean']=calyearly['Energy'].mean()
models.at[model_name,'CalYearStDev']=calyearly['Energy'].std()
models.at[model_name,'CalYearCoefVar']=models.at[model_name,'CalYearStDev']/models.at[model_name,'CalYearMean']



if dryrun==False:
    models.to_csv(input_data + "models.csv")

models

